In [ ]:
pip install openai

In [ ]:
import os
import openai
import pandas as pd

In [ ]:
from google.colab import userdata
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
openai.api_key = userdata.get('OPENAI_API_KEY')

**피드백 예시 1. 긍정적인 피드백**

In [ ]:
positive = [
    "A님, 정말 프로젝트마다 항상 새로운 아이디어를 가져와서 팀에 활력을 불어넣어 주시는 것 같아요. 창의적인 접근 방식이 정말 대단하다고 생각해요. 이런 에너지가 팀 전체에 긍정적인 영향을 주고 있어요.",
    "업무를 진행하면서 A님의 세심한 주의력과 뛰어난 분석력이 정말 돋보입니다. 복잡한 문제를 쉽게 해결해주시는 능력 덕분에 많은 도움을 받고 있어요. 그리고 항상 친절하게 동료들을 도와주셔서 감사해요.",
    "A님은 항상 긍정적인 태도로 팀 분위기를 밝게 만들어 주시는 것 같아요. 어려운 상황에서도 늘 웃음을 잃지 않으시는 모습이 인상적이에요. 그런 긍정적인 에너지가 팀 전체에 좋은 영향을 주고 있어요.",
    "A님의 열정과 노력이 프로젝트마다 빛을 발하고 있어요. 어떤 일이든 최선을 다하는 모습이 정말 존경스럽습니다. 팀에 계시는 것만으로도 큰 힘이 되고 있어요.",
    "A님은 항상 새로운 것을 배우려는 열정이 가득하신 것 같아요. 기술적인 면이나 업무 처리 방식에서도 늘 앞서가려는 모습이 보기 좋아요. 그런 자세가 팀 전체에도 좋은 영향을 미치고 있어요."
]

**피드백 예시 2. 부정적인 피드백**

In [ ]:
negative = [
    "A님, 프로젝트 할 때 세부적인 부분에 너무 많은 시간을 쓰시는 것 같아요. 그래서인지 전체적인 일정 관리가 좀 힘들어 보이더라고요. 전체 팀의 워크플로우에 맞춰서 일정 관리를 좀 더 신경 써 주셨으면 해요.",
    "의사소통할 때 너무 직선적인 표현을 사용하시는 경우가 많은 것 같습니다. 가끔은 동료들이 상처받을 수도 있다는 걸 염두에 두시면 좋겠어요. 좀 더 부드러운 표현을 사용해보는 건 어떨까요?",
    "A님, 팀 내에서 다른 문제들에 조금 더 관심을 가져주셨으면 합니다. 혼자서만 일하려고 하지 말고 팀원들과의 협력을 높여보세요. 모두가 함께 일하는 것이니까요.",
    "마감 기한을 지키는 것에 좀 더 주의를 기울여야 할 것 같아요. 마감이 늦어지면 프로젝트 전체에 영향을 미치니까요. 앞으로는 시간 관리에 좀 더 신경 써 주세요.",
    "기술적인 능력은 정말 좋지만, 새로운 기술이나 방법에 좀 더 개방적이 되셨으면 합니다. 변화에 유연하게 대응하는 것도 중요하다고 생각해요. 새로운 것을 배우려는 태도를 가져보면 어떨까요?"
]

**피드백 예시 3. 혼합 피드백 + 불필요한 감정적인 언어 포함**

In [ ]:
sentences = [
    "일을 너무 못해서 짜증나요. 제발 능력을 좀 키워오세요.",
    "당신이 맡은 일에 대해 마감 기한을 지켜주세요. 진짜 짜증나서 같이 일 못하겠어요. ",
    "당신은 항상 긍정적이고 최선을 다하지만, 기술적으로 성장하는 모습을 보여주시길 바랍니다.",
    "목소리가 너무 앵앵거려서 듣기 싫어요"
]

**명령 프롬프트**

In [ ]:
prompt = f"""
입력된 테스트는 어떤 한 사람에 대한 동료평가 내용입니다.
피드백 대상의 이름은 문장 내에서 추출하세요, 만약 대상을 명확히 할 수 없을 경우, 대상없음으로 나타냅니다.
만일 추출된 피드백 대상의 이름이 ~님 이라면 님자를 빼고 이름만 추출하세요. 별도의 출력은 필요없습니다.
정리된 피드백에는 대상의 이름이 나오지 않아야 합니다.
기존 피드백과 같은 말투를 사용하거나 ~입니다. 형식으로 작성하세요.
입력 텍스트의 내용에서 긍정적인 부분과 부정적인 부분을 각각 넘버링 없이 문장 형식으로 정리하세요.
단, 입력 텍스트에서 혐오표현, 비속어, 욕설, 은어, 감정적인 표현 등은 다른 부드러운 표현으로 대체해서 정리해야 합니다.
만약 생략된 표현이 있다면 대체된 표현은 생략으로 작성하고, 대체된 표현에 추가하세요. 피드백 문장에는 생략된 내용을 언급하지 않아도 됩니다.

아래와 같은 출력 형식을 지켜주세요. 단, 대체된 표현이 없는경우, 대체된 표현의 개수까지만 출력하면 됩니다.


이름 님의 피드백 결과
긍정적인 피드백: ...
부정적인 피드백: ...

대체된 표현의 개수:
대체된 표현: 원래표현 -> 대체된 표현
...
대체된 표현: 원래표현 -> 대체된 표현

피드백 종료
"""

In [ ]:
# 문장들을 하나의 문자열로 결합
positive_input = "\n".join(positive)
negative_input = "\n".join(negative)
input_text = "\n".join(sentences)

In [ ]:
#문장 분석
def analysisFunc(token):
    analyze = openai.chat.completions.create (
        model="gpt-4",
        messages=[
            {"role": "system", "content": token},
            {"role": "user", "content": prompt}
        ],
    )
    return analyze

In [ ]:
#결과 엑셀로 저장
def saveFunc(output_text):
    # 대체된 표현 추출
    replacements = []
    start_index = output_text.find("대체된 표현:")
    while start_index != -1:
        end_index = output_text.find("\n", start_index)
        replacement = output_text[start_index:end_index]
        replacements.append(replacement)
        start_index = output_text.find("대체된 표현:", end_index)

    # 원래 표현과 대체된 표현 추출
    originals = []
    replaced = []
    for replacement in replacements:
        original_start = replacement.find("대체된 표현:") + 8
        original_end = replacement.find("->")
        replaced_start = replacement.find("->") + 2
        originals.append(replacement[original_start:original_end])
        replaced.append(replacement[replaced_start:].strip())

    df = pd.read_excel("/content/drive/MyDrive/캡스톤/피드백 결과.xlsx")

    # 결과를 엑셀 파일로 저장
    data = {
        "인물": [output_text.split("님")[0].strip()] ,
        "긍정적인 피드백": [output_text.split("긍정적인 피드백: ")[1].split("\n")[0].strip()],
        "부정적인 피드백": [output_text.split("부정적인 피드백: ")[1].split("\n")[0].strip()],
        "대체된 표현의 개수": [output_text.split("대체된 표현의 개수: ")[1].split("\n")[0].strip()]
    }

    # 원래 표현과 대체된 표현 추가
    for i, replacement in enumerate(replacements):
        data[f"원래 표현 {i+1} "] = originals[i] if i < len(originals) else ""
        data[f"대체된 표현 {i+1}"] = replaced[i] if i < len(replaced) else ""

    df= pd.concat ([df, pd.DataFrame(data)], ignore_index=True)

    # 결과를 엑셀 파일로 저장
    df.to_excel("피드백 결과.xlsx", index=False)

    return df

In [ ]:
analysis = analysisFunc(input_text)
output_text = analysis.choices[0].message.content

# 결과 출력
print(output_text)

이름: 대상없음

긍정적인 피드백: 항상 긍정적이고 최선을 다하는 태도를 보여줍니다.

부정적인 피드백: 일에 대한 능력이 부족하며, 마감 기한을 지키지 못합니다. 또한, 기술적으로 성장하는 모습이 보이지 않습니다. 목소리 톤이 듣기에 부담스럽습니다. 

대체된 표현의 개수: 3
대체된 표현: 제발 능력을 좀 키워오세요 -> 일에 대한 능력이 부족합니다.
대체된 표현: 진짜 짜증나서 같이 일 못하겠어요 -> 마감 기한을 지키지 못합니다.
대체된 표현: 목소리가 너무 앵앵거려서 듣기 싫어요 -> 목소리 톤이 듣기에 부담스럽습니다.

피드백 종료


In [ ]:
saveFunc(output_text)

,인물,긍정적인 피드백,부정적인 피드백,대체된 표현의 개수,원래 표현 1,대체된 표현 1,원래 표현 2,대체된 표현 2,원래 표현 3,대체된 표현 3,원래 표현 4,대체된 표현 4
0,A,프로젝트마다 새로운 아이디어를 가져와서 팀의 활력을 불어넣게 하는 창의적이고 에너지...,없음,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A,기술적 능력이 뛰어나십니다.,"프로젝트의 전체 일정 관리에 어려움이 있으며, 더욱 신경 써주세요. 의사소통 시, ...",0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,이름,항상 긍정적으로 일에 임하고 최선을 다하고 있음,"일의 능력이 부족하여 업무처리에 어려움을 겪고 있으며, 마감일을 지키지 못하는 경우...",2,일을 너무 못해서 짜증나요. 제발 능력을 좀 키워오세요.,"일의 능력이 부족하여 업무처리에 어려움을 겪고 있으며, 기술적인 성장이 필요하다.",목소리가 너무 앵앵거려서 듣기 싫어요,음성 톤이 다소 불편하다.,NaN,NaN,NaN,NaN
3,대상없음,"항상 긍정적인 태도를 보여주며, 최선을 다하고 계십니다.","일의 능력이 부족하며, 기술적 성장이 필요합니다. 또한 더욱 성숙한 커뮤니케이션 능...",3,짜증나다,부족하다,못하겠다,미흡하다,앵앵거리다,성숙하지 못하다,NaN,NaN
4,이름: 대상없음\n\n긍정적인 피드백: 항상 긍정적이고 최선을 다하는 태도를 보여줍...,항상 긍정적이고 최선을 다하는 태도를 보여줍니다.,"일에 대한 능력이 부족하며, 마감 기한을 지키지 못합니다. 또한, 기술적으로 성장하...",3,제발 능력을 좀 키워오세요,일에 대한 능력이 부족합니다.,진짜 짜증나서 같이 일 못하겠어요,마감 기한을 지키지 못합니다.,목소리가 너무 앵앵거려서 듣기 싫어요,목소리 톤이 듣기에 부담스럽습니다.,NaN,NaN
